In [2]:
import rasterio
import geopandas as gpd
import pandas as pd
import os

# Directory containing your point shapefiles
points_dir = './Centroids_data'
# Output folder for the CSV files
output_folder = 'Temporal_series_data'

# Check if the output folder exists, create it if it does not
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Pre-load all raster data sources outside of the shapefile loop to avoid reopening them
rasters = {}
for year in range(2012, 2022):  # From 2012 to 2021
    image_path = f'Images/{year}.tif'  # Update this path if necessary
    rasters[year] = rasterio.open(image_path)

# Keep track of the filenames
csv_filenames = []

# Iterate over all shapefiles in the points directory
for shp_file in sorted(os.listdir(points_dir)):
    if shp_file.endswith('.shp'):
        pixel_data = []  # Initialize a list to store pixel value data

        shp_path = os.path.join(points_dir, shp_file)
        points_gdf = gpd.read_file(shp_path)  # Load the shapefile

        # Collect all point coordinates
        coordinates = [(point.geometry.x, point.geometry.y) for index, point in points_gdf.iterrows()]

        # For each raster year
        for year, src in rasters.items():
            # Sample all points at once
            values = list(src.sample(coordinates))
            
            # Append pixel data for each point
            for i, val in enumerate(values):
                pixel_data.append({
                    'Date': year,
                    'ID': points_gdf.iloc[i]['ID'],  # Assuming 'ID' is the column name
                    'Blue': val[0],
                    'Green': val[1],
                    'Red': val[2],
                    'NIR': val[3]  # Assuming band order is Blue, Green, Red, NIR
                })

        pixel_df = pd.DataFrame(pixel_data)  # Convert to DataFrame
        csv_filename = os.path.join(output_folder, f"{shp_file.split('.')[0]}.csv")
        pixel_df.to_csv(csv_filename, index=True)  # Save to CSV
        csv_filenames.append(csv_filename)  # Store the filename

# Sort the filenames list
csv_filenames.sort()  # Sorts filenames alphabetically, adjust if a different sorting is required

# Optionally rename the files based on sorted order
for i, filename in enumerate(csv_filenames, start=1):
    new_filename = os.path.join(output_folder, f"tile_{i}.csv")
    os.rename(filename, new_filename)  # Rename the file

# Close raster files
for src in rasters.values():
    src.close()



KeyboardInterrupt

